In [14]:
%pip install cassiopeia
%pip install mysql.connector

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for pymysql from https://files.pythonhosted.org/packages/e5/30/20467e39523d0cfc2b6227902d3687a16364307260c75e6a1cb4422b0c62/PyMySQL-1.1.0-py3-none-any.whl.metadata
  Using cached PyMySQL-1.1.0-py3-none-any.whl.metadata (4.4 kB)
Using cached PyMySQL-1.1.0-py3-none-any.whl (44 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cassiopeia as cass
import mysql.connector as Mc
from cassiopeia.data import Queue, Position, GameMode, Side, Lane, Tier
from cassiopeia.core import Summoner, Match
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from cassiopeia import Champions, Champion
from fonct_mysql import *

In [6]:
cass.set_riot_api_key(key="RGAPI-32acba9c-c213-44d7-9b9e-a826d0419ca0")

In [9]:
def Update_summoners_name(summoner_name: str, region: str):
    summoner_name = Summoner(name = summoner_name, region = region)
    entries = summoner_name.league_entries
    count = 0
    try:
        for position, entry in enumerate(entries.fives.league.entries):
            
            req_ins_two('`summoners` (`id_summoner`, `name_summoner`, `riot_id_summoner`)',  entry.summoner.name, entry.summoner.id, password= "")
            print(f"Add {entry.summoner.name} to the database...")
            count +=1

    except ValueError:
        print("Try with a other name of summoner")
    print(f"Add {count} summoners to the database")

In [42]:
Update_summoners_name("Targamas", "EUW")

Making call: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Targamas
Making call: https://euw1.api.riotgames.com/lol/league/v4/entries/by-summoner/5SZJ3-WIDJdPqaG_oRqe1rxXiIPHvUPt-sAFYEZLwe9-xb4
Making call: https://euw1.api.riotgames.com/lol/league/v4/leagues/d0944f91-f5f6-4372-8437-9ccebf73a30f
Add Poppystar to the database...
Add Kannonbiraki to the database...
Add Triewso to the database...
Add remember l to the database...
Add rivotril3000 to the database...
Add ITZY Yejì to the database...
Add Worlds Best TF to the database...
Add AI SurvivalPure to the database...
Add zxcasdzxc to the database...
Add ventos to the database...
Add Aschmeday to the database...
Add Itsjustaprankbr to the database...
Add 1Shockwave2 to the database...
Add sαku to the database...
Add KogNBardTorture to the database...
Add DFF Kuro to the database...
Add Müciioka to the database...
Add  to the database...
Add Lightning Thunde to the database...
Add adrii GORDO to the database...
Add 

In [10]:
def s_c(p, columns):
    for i in p:
        if i.side == Side.red and i.lane == Lane.top_lane and columns == "red_top":
            return i.champion.name
        elif i.side == Side.red and i.lane == Lane.jungle and columns == "red_jun":
            return i.champion.name
        elif i.side == Side.red and i.lane == Lane.mid_lane and columns == "red_mid":
            return i.champion.name
        elif i.side == Side.red and i.lane == Lane.bot_lane and columns == "red_bot":
            return i.champion.name
        elif i.side == Side.red and i.lane == Lane.utility and columns == "red_sup":
            return i.champion.name
        elif i.side == Side.blue and i.lane == Lane.top_lane and columns == "blu_top":
            return i.champion.name
        elif i.side == Side.blue and i.lane == Lane.jungle and columns == "blu_jun":
            return i.champion.name
        elif i.side == Side.blue and i.lane == Lane.mid_lane and columns == "blu_mid":
            return i.champion.name
        elif i.side == Side.blue and i.lane == Lane.bot_lane and columns == "blu_bot":
            return i.champion.name
        elif i.side == Side.blue and i.lane == Lane.utility and columns == "blu_sup":
            return i.champion.name

In [11]:
def df_matchs(match, p, ref_id_summoner):
    parametres = {
        "red_top": s_c(p, "red_top"),
        "red_jun": s_c(p, "red_jun"),
        "red_mid": s_c(p, "red_mid"),
        "red_bot": s_c(p, "red_bot"),
        "red_sup": s_c(p, "red_sup"),
        "blu_top": s_c(p, "blu_top"),
        "blu_jun": s_c(p, "blu_jun"),
        "blu_mid": s_c(p, "blu_mid"),
        "blu_bot": s_c(p, "blu_bot"),
        "blu_sup": s_c(p, "blu_sup"),
        "win_side": match.blue_team.win,
        "ref_id_summoners" : ref_id_summoner,
        "riot_id_match": match.id
    }

    df = pd.DataFrame(parametres, index=[0])
    return df

In [12]:
def check_participants(p):
    try:
        for i in p:
            if i.lane == None:
                return False
            else:
                pass
    except KeyError:
        return False

In [13]:
def get_matches(summoner, num_matches):
    match_dataframes = []
    count = 0
    for i , match in enumerate(summoner.match_history[1:num_matches]):

        vr = req_one(order= 'SELECT `riot_id_match` FROM `matches` WHERE `riot_id_match` = %s', value_ord= (match.id, ), password= "")

        p = match.participants

        cp = check_participants(p)

        if vr != None:
            continue
        if count >= 10:
            break
        if cp == False:
            continue
        if match.start == False:
            print("Match not begin...")
            continue
        if match.mode != GameMode.classic:
            print("Not Classic Mode...")
            continue
        if match.queue != Queue.ranked_solo_fives:
            print("Not Ranked five...")
            continue
        if match.is_remake == True:
            print("Remake match...")
            continue

        ref_id_summoner = req_one(order='SELECT `id_summoner` FROM `summoners` WHERE `riot_id_summoner`= %s', value_ord= (summoner.id,), password="")

        df_match = df_matchs(match, p, ref_id_summoner)
        match_dataframes.append(df_match)
        count += 1

    return match_dataframes
    
def main(id : str , region : str):
    try:
        summoner = Summoner(id = id , region=region)
        match_dataframes = get_matches(summoner, 50)

        df_main = pd.concat(match_dataframes, ignore_index= True)

        req_df(dataframe= df_main, table='matches', password="")
            
    except KeyError:
        print("Not enough game...")
    except ValueError:
        print("Empty Data...")

In [14]:
def all_main(region : str):
    all_sum = req_all('SELECT `riot_id_summoner` FROM `summoners`', value_ord=None, password="")
    for i in all_sum:
        ref_is = req_one('SELECT `id_summoner` FROM `summoners` WHERE `riot_id_summoner` = %s', value_ord= i, password="")
        count_m = req_one('SELECT COUNT(*) FROM `matches` WHERE `ref_id_summoners` = %s', value_ord= ref_is, password= "")
        if count_m[0] >= 10:
            continue
        else:
            main(i[0], region)

In [50]:
test_all = all_main("EUW")

Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/xYE1L8bCYV2cWV2XDgSauAUH4OSUlZ5eQebaOcWudX637GSlSUHadZBewe51e_k-shDvEF2IjpydPw/ids?start=0&count=100
Not enough game...
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/7gSU69oh-LM1e_1Cb52K2AzEz6IEFJYtFUOm9ZZpLnnzSOujNYEOK0o67Gh8bmfBwxSddElm8S6Ykw/ids?start=0&count=100
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...
Not Ranked five...


In [36]:
def df_champions(champion):
    parametres = {
        "id_champion" : champion.id,
        "name_champ" : champion.name
    }
    
    df = pd.DataFrame(parametres, index=[0])
    return df
    
def champion_stats(region : str):
    cham_dataframe = []
    champions = Champions(region=region)
    for champion in champions:
        df_ch = df_champions(champion)
        cham_dataframe.append(df_ch)
    final_df = pd.concat(cham_dataframe, ignore_index=True)
    req_df(dataframe= final_df, table= 'champions_list', password="")

In [37]:
champion_stats("EUW")